# QENIE (定量内分泌网络互作估计) - 主分析流程

## 项目简介

QENIE (Quantitative Endocrine Network Interaction Estimation) 是一个基于相关性的方法，用于识别内分泌互作关系。

## 生物学背景

内分泌系统通过分泌蛋白（激素、细胞因子等）在不同组织间传递信号。本分析流程旨在：

1. **识别跨组织的基因表达相关性**：分析肝脏和脂肪组织之间的基因表达模式
2. **筛选分泌蛋白**：从所有基因中筛选出能够被分泌到细胞外的蛋白质
3. **计算显著性评分（Ssec）**：量化每个分泌蛋白与目标组织基因的相关性强度
4. **通路富集分析**：识别被特定分泌蛋白调控的生物学通路

## 计算方法说明

本notebook使用以下统计方法：
- **双权重中值相关性（bicor）**：一种稳健的相关性计算方法，对异常值不敏感
- **P值转换**：使用-log(p值)放大显著性差异
- **归一化处理**：按目标组织基因数量归一化，确保不同组织间可比性

## 示例分析

本notebook演示**肝脏→脂肪**组织回路的分析，识别肝脏分泌的、影响脂肪组织的蛋白质。

## 第一步：环境准备与库导入

### 所需R包说明

- **WGCNA** (Weighted Gene Co-expression Network Analysis): 加权基因共表达网络分析包
  - 提供bicor函数：计算双权重中值相关性
  - 比Pearson相关系数更稳健，对异常值不敏感
  
- **reshape2**: 数据重塑工具
  - melt函数：将宽格式数据转为长格式
  - 便于后续数据处理和可视化
  
- **tidyverse**: R语言现代数据科学工具集
  - dplyr: 数据操作（filter、select、mutate、arrange等）
  - ggplot2: 数据可视化
  - 提供管道操作符%>%，使代码更易读

In [ ]:
# 加载所需的R包
# 如果未安装，请先运行：install.packages(c("WGCNA", "reshape2", "tidyverse"))

library(WGCNA)
library(reshape2)
library(tidyverse)

## 第二步：数据导入

### 数据文件说明

#### 1. Secreted_proteins_Uniprot.txt
- **来源**: UniProt数据库 (www.uniprot.org)
- **内容**: 小鼠（Mus musculus）中所有已注释的分泌蛋白
- **筛选条件**: 
  - 物种：Mus musculus [10090]
  - 亚细胞定位：Secreted [SL-0243]
- **生物学意义**: 分泌蛋白可以从细胞释放，在组织间传递信号，是内分泌和旁分泌调控的关键分子

#### 2. adipose.txt (脂肪组织表达数据)
- **数据来源**: GEO数据库 GSE64770
- **实验设计**: 来自HMDP (Hybrid Mouse Diversity Panel) 的106个小鼠品系
- **数据处理**:
  - 平台：Affymetrix HT_MG-430A芯片
  - RMA归一化后的表达数据
  - 多个探针的基因取平均值
  - 同品系小鼠取平均值
- **数据维度**: 12,242个基因 × 106个品系
- **生物学背景**: 脂肪组织是重要的内分泌器官，分泌脂肪因子影响全身代谢

#### 3. liver.txt (肝脏组织表达数据)
- **数据来源**: 同上 GSE64770
- **数据维度**: 12,242个基因 × 106个品系
- **生物学背景**: 肝脏是主要的代谢器官，分泌多种激素和细胞因子调控代谢稳态

### 数据格式
- **行**：不同的小鼠品系（遗传背景）
- **列**：不同的基因
- **值**：基因表达水平（RMA归一化后的对数转换值）

In [ ]:
# 导入分泌蛋白列表
# header = T: 文件包含列名
# check.names = F: 保持原始列名，不自动转换特殊字符
Secreted_proteins <- read.delim("Secreted_proteins_Uniprot.txt", header = T, check.names=F)

# 查看分泌蛋白数据的前几行
print("分泌蛋白列表预览：")
head(Secreted_proteins)

# 统计分泌蛋白的数量
print(paste("分泌蛋白总数：", nrow(Secreted_proteins)))

In [ ]:
# 导入脂肪组织表达数据
# check.names = F: 保持基因名称不被R自动修改
Adipose <- read.delim('adipose.txt', check.names=F)

print("脂肪组织表达数据维度：")
print(paste("品系数量：", nrow(Adipose)))
print(paste("基因数量：", ncol(Adipose)))

# 查看数据的前几行和前几列
print("\n脂肪组织表达数据预览：")
Adipose[1:5, 1:5]

In [ ]:
# 导入肝脏组织表达数据
Liver <- read.delim('liver.txt', check.names=F)

print("肝脏组织表达数据维度：")
print(paste("品系数量：", nrow(Liver)))
print(paste("基因数量：", ncol(Liver)))

# 查看数据的前几行和前几列
print("\n肝脏组织表达数据预览：")
Liver[1:5, 1:5]

## 第三步：构建跨组织相关性和P值矩阵

### 生物学意义

如果肝脏中某个基因的表达水平与脂肪组织中另一个基因的表达水平在不同遗传背景（不同小鼠品系）下呈现相关性，这可能表明：
1. 肝脏基因产物（如果是分泌蛋白）可能通过血液循环影响脂肪组织基因的表达
2. 两个基因可能受共同的遗传或环境因素调控
3. 存在潜在的内分泌调控关系

### 计算方法：bicorAndPvalue

#### bicor（双权重中值相关性）的优势
- **稳健性**：对异常值不敏感，比Pearson相关系数更可靠
- **原理**：通过加权降低极端值的影响
- **适用场景**：基因表达数据常含异常值，bicor更适合

#### 参数说明
- **Liver**: 源组织（肝脏）的基因表达矩阵
- **Adipose**: 目标组织（脂肪）的基因表达矩阵
- **use='pairwise.complete.obs'**: 每对基因的相关性计算只使用两者都有数据的样本
  - 处理缺失值的策略
  - 保证每次相关性计算使用的样本集合可能不同，但都是完整的

#### 输出结果
- **liv.adip$bicor**: 相关系数矩阵
  - 行：肝脏基因
  - 列：脂肪基因
  - 值：-1到1之间，表示相关性强度和方向
    - 接近1：正相关（肝脏基因高表达时，脂肪基因也高表达）
    - 接近-1：负相关（肝脏基因高表达时，脂肪基因低表达）
    - 接近0：无相关性
    
- **liv.adip$p**: P值矩阵
  - 相同维度
  - 表示每个相关性的统计显著性
  - P值越小，相关性越显著（越不可能是随机产生的）

### 计算复杂度
- 矩阵维度：12,242 × 12,242 = 约1.5亿个相关性
- 计算时间：可能需要几分钟到十几分钟
- 内存需求：需要足够的RAM存储相关性矩阵

In [ ]:
# 计算肝脏和脂肪组织间的跨组织相关性及其显著性P值
# 这一步是整个分析的核心，计算所有肝脏-脂肪基因对的相关性
# 注意：这个计算可能需要较长时间（几分钟到十几分钟）

print("开始计算跨组织相关性矩阵...")
print("这可能需要几分钟时间，请耐心等待...")

# 计算双权重中值相关性和P值
liv.adip = bicorAndPvalue(Liver, Adipose, use='pairwise.complete.obs')

print("\n计算完成！")
print("\n相关性矩阵维度：")
print(paste("肝脏基因数：", nrow(liv.adip$bicor)))
print(paste("脂肪基因数：", ncol(liv.adip$bicor)))
print(paste("总相关性对数：", nrow(liv.adip$bicor) * ncol(liv.adip$bicor)))

# 查看相关性矩阵的一小部分
print("\n相关性矩阵预览（前5×5）：")
liv.adip$bicor[1:5, 1:5]

## 第四步：计算Ssec显著性评分

### 评分方法：Ssec (Significance Score)

#### 计算公式
```
原始score = Σ(-log(p值))  # 对每个肝脏基因，累加其与所有脂肪基因相关性的-log(p值)
Ssec = 原始score / 脂肪组织基因数量  # 归一化处理
```

#### 计算原理

1. **为什么使用-log(p值)？**
   - P值本身很小（如0.001、0.0001），不便于累加比较
   - -log转换后变为正数（如3、4），且P值越小转换值越大
   - 数学特性：-log(0.001) ≈ 3, -log(0.0001) ≈ 4
   - 放大显著性差异，使高度显著的相关性贡献更大

2. **为什么对每行求和（rowSums）？**
   - 每一行代表一个肝脏基因
   - 求和表示该肝脏基因与所有脂肪基因相关性的累积显著性
   - 得分高的基因：与脂肪组织基因表达模式整体关联性强

3. **为什么要归一化（除以基因数量）？**
   - 不同研究或数据集可能包含不同数量的基因
   - 归一化确保不同数据集间的Ssec值可比较
   - 避免基因数量多的数据集自动获得更高分数

#### 生物学解释

**Ssec值高的基因代表什么？**
- 该肝脏基因的表达模式与脂肪组织的整体基因表达模式高度相关
- 如果该基因编码分泌蛋白，可能通过血液循环广泛影响脂肪组织的基因表达
- 可能是重要的内分泌调控因子

#### 筛选标准

只保留**分泌蛋白**，因为：
1. 只有分泌蛋白能离开细胞，通过血液到达远端组织
2. 非分泌蛋白即使相关性高，也不能直接作用于其他组织
3. 这是识别内分泌互作的关键筛选步骤

In [ ]:
# 步骤4.1: 计算每个肝脏基因的原始显著性评分
# rowSums: 对矩阵的每一行求和
# -log(liv.adip$p): 对P值矩阵取负对数
#   - P值越小（越显著），-log(p)越大
#   - 例如：p=0.01时，-log(0.01)≈2; p=0.001时，-log(0.001)≈3

print("计算显著性评分...")
scores = rowSums(-log(liv.adip$p))

print("\n原始评分统计：")
print(paste("基因总数：", length(scores)))
print(paste("最高分：", round(max(scores), 2)))
print(paste("最低分：", round(min(scores), 2)))
print(paste("平均分：", round(mean(scores), 2)))
print(paste("中位数：", round(median(scores), 2)))

# 查看评分最高的前10个基因
print("\n原始评分最高的前10个基因：")
head(sort(scores, decreasing = TRUE), 10)

In [ ]:
# 步骤4.2: 筛选分泌蛋白并归一化
# 使用tidyverse的管道操作（%>%）使代码更清晰易读

print("筛选分泌蛋白并计算归一化Ssec评分...")

Ssec = data.frame(Gene_symbol = names(scores), score = scores) %>%
  # filter: 只保留在分泌蛋白列表中的基因
  # %in%: 检查Gene_symbol是否存在于分泌蛋白的主基因名列表中
  filter(Gene_symbol %in% Secreted_proteins$`Gene names  (primary )`) %>%
  
  # mutate: 创建新列或修改现有列
  # 计算Ssec = 原始score / 脂肪组织基因数量
  # 这个归一化步骤很重要，确保不同研究间的可比性
  mutate(Ssec = score / length(colnames(Adipose))) %>%
  
  # select: 选择要保留的列（去掉中间计算用的score列）
  select(-score) %>%
  
  # arrange: 按Ssec降序排列
  # desc(): 降序排列函数
  arrange(desc(Ssec))

print("\n筛选结果统计：")
print(paste("分泌蛋白数量：", nrow(Ssec)))
print(paste("最高Ssec：", round(max(Ssec$Ssec), 4)))
print(paste("最低Ssec：", round(min(Ssec$Ssec), 4)))
print(paste("平均Ssec：", round(mean(Ssec$Ssec), 4)))

# 显示排名前20的分泌蛋白
print("\n排名前20的分泌蛋白及其Ssec评分：")
head(Ssec, 20)

In [ ]:
# 步骤4.3: 保存结果到文件
# 这个文件包含所有分泌蛋白的Ssec评分，按显著性排序

output_file = "Liver X Adipose ranked by sig score"
write.table(Ssec, 
            file = output_file, 
            row.names = F,      # 不保存行名
            col.names = T,      # 保存列名
            sep = '\t',         # 使用制表符分隔
            quote = F)          # 不给字符串加引号

print(paste("结果已保存到文件：", output_file))
print("\n文件包含两列：")
print("1. Gene_symbol: 基因符号")
print("2. Ssec: 归一化的显著性评分")

### Ssec结果解读

#### 重要发现

根据原始分析，**Notum**基因在筛选分泌蛋白后排名第5（Ssec ≈ 4.148）。

#### Notum的生物学背景

- **功能**：Notum是一种分泌型脂肪酶
- **作用机制**：抑制Wnt信号通路
- **代谢作用**：
  - 调控脂肪组织的脂质代谢
  - 影响胰岛素敏感性
  - 参与能量平衡调控

#### 组织特异性验证（可选步骤）

原始流程中提到使用**BioGPS**数据库（biogps.org）进行组织特异性验证：
- 确认基因在源组织（肝脏）中高表达
- 确认基因在目标组织（脂肪）中低表达或不表达
- 这符合内分泌信号传递的模式：源组织产生，目标组织响应

通过组织特异性验证后，Notum在最终排名中上升到第2位，这增加了其作为肝脏-脂肪内分泌因子的可信度。

## 第五步：条件相关性矩阵用于通路富集分析

### 分析目标

识别特定分泌蛋白（如Notum）在目标组织（脂肪）中调控的生物学通路和功能模块。

### 分析策略

#### 为什么需要"条件"相关性？

我们要找出：**当肝脏中Notum表达改变时，脂肪组织中哪些基因的表达也会改变**

#### 方法步骤

1. **提取Notum的相关性向量**
   - 从完整相关性矩阵中提取Notum与所有脂肪基因的相关系数
   - 移除Notum自己（避免自相关）

2. **按相关性方向分类**
   - **正相关基因**：Notum高表达时这些基因也高表达
     - 可能是Notum"上调"或"激活"的通路
   - **负相关基因**：Notum高表达时这些基因低表达
     - 可能是Notum"下调"或"抑制"的通路
   - **绝对值相关基因**：无论方向，找出相关性最强的基因
     - 找出Notum"影响最大"的通路

3. **选择Top基因用于通路富集**
   - 通常选择Top 500个基因
   - 这个数量适合多数通路富集分析工具
   - 平衡了统计效力和生物学相关性

### 下游分析

生成的基因列表可以用于：
- **KEGG通路富集**：识别相关的代谢通路
- **GO富集分析**：识别生物学过程、分子功能、细胞组分
- **Reactome通路分析**：详细的生物化学通路
- **自定义通路数据库**：特定研究领域的通路集

### 示例：Notum的分析

以Notum为例，我们将：
1. 提取Notum与脂肪基因的相关性
2. 生成三个基因列表文件
3. 这些文件可以直接用于通路富集分析工具

In [ ]:
# 步骤5.1: 将相关性矩阵从宽格式转为长格式
# 这样便于后续的筛选和操作

print("重塑相关性矩阵为长格式...")

# melt: 将宽格式矩阵转为长格式数据框
# 原始矩阵：行是肝脏基因，列是脂肪基因
# 转换后：每一行代表一个肝脏-脂肪基因对及其相关系数
bicor.data = melt(liv.adip$bicor)

# 重命名列名，使其更有意义
colnames(bicor.data) = c('Gene_symbol_1',  # 肝脏基因
                         'Gene_symbol_2',   # 脂肪基因
                         'bicor')           # 相关系数

print("\n长格式数据维度：")
print(paste("总行数（基因对数）：", nrow(bicor.data)))

# 显示数据结构
print("\n长格式数据预览：")
head(bicor.data, 10)

In [ ]:
# 步骤5.2: 提取Notum基因的相关性数据
# 这里以Notum为例，你可以替换为任何感兴趣的基因

target_gene = 'Notum'
print(paste("提取", target_gene, "基因的相关性数据..."))

# 筛选出Gene_symbol_1为Notum的所有行
# 这些行包含Notum与所有脂肪基因的相关性
Notum = filter(bicor.data, Gene_symbol_1 == target_gene) %>%
  # 移除Gene_symbol_1列（因为都是Notum，不需要了）
  select(-Gene_symbol_1) %>%
  # 重命名Gene_symbol_2为Gene_symbol
  rename(Gene_symbol = Gene_symbol_2)

print("\nNotum相关性数据统计：")
print(paste("相关基因总数：", nrow(Notum)))
print(paste("最大正相关：", round(max(Notum$bicor), 4)))
print(paste("最大负相关：", round(min(Notum$bicor), 4)))
print(paste("平均相关性：", round(mean(Notum$bicor), 4)))

# 显示相关性最强的前10个基因（正相关）
print("\n与Notum正相关最强的前10个脂肪基因：")
Notum %>% arrange(desc(bicor)) %>% head(10)

In [ ]:
# 步骤5.3: 生成正相关基因列表（Notum可能上调的通路）

print("生成正相关基因列表...")

# 选择与Notum正相关最强的前500个基因
# 这些基因在Notum高表达时可能也会高表达
# 代表Notum可能"激活"或"增强"的生物学通路
upreg = Notum %>% 
  arrange(desc(bicor)) %>%  # 按相关系数降序排列
  head(500)                  # 取前500个

# 保存到文件，用于通路富集分析
output_file = "Positive Notum Liver X Adipose Pathways Enrichment File"
write.table(upreg, 
            file = output_file, 
            col.names = F,  # 不保存列名（通路富集工具通常不需要）
            sep = '\t', 
            quote = F)

print(paste("正相关基因列表已保存到：", output_file))
print("\n正相关基因Top 10：")
head(upreg, 10)

In [ ]:
# 步骤5.4: 生成负相关基因列表（Notum可能下调的通路）

print("生成负相关基因列表...")

# 选择与Notum负相关最强的前500个基因
# 这些基因在Notum高表达时可能会低表达
# 代表Notum可能"抑制"或"下调"的生物学通路
downreg = Notum %>% 
  arrange(bicor) %>%  # 按相关系数升序排列（最负的在前）
  head(500)

output_file = "Negative Notum Liver X Adipose Pathways Enrichment File"
write.table(downreg, 
            file = output_file, 
            col.names = F, 
            sep = '\t', 
            quote = F)

print(paste("负相关基因列表已保存到：", output_file))
print("\n负相关基因Top 10：")
head(downreg, 10)

In [ ]:
# 步骤5.5: 生成绝对值相关基因列表（Notum影响最大的通路）

print("生成绝对值相关基因列表...")

# 选择与Notum相关性绝对值最大的前500个基因
# abs(): 取绝对值
# 无论正负，只要相关性强就选择
# 代表Notum"影响最大"的生物学通路，不考虑调控方向
totalreg = Notum %>% 
  arrange(desc(abs(bicor))) %>%  # 按相关系数绝对值降序排列
  head(500)

output_file = "Absolute Notum Liver X Adipose Pathways Enrichment File"
write.table(totalreg, 
            file = output_file, 
            col.names = F, 
            sep = '\t', 
            quote = F)

print(paste("绝对值相关基因列表已保存到：", output_file))
print("\n绝对值相关基因Top 10：")
head(totalreg, 10)

## 分析总结

### 主要输出文件

1. **Liver X Adipose ranked by sig score**
   - 所有分泌蛋白的Ssec评分
   - 用于识别候选内分泌因子
   - 按显著性排序

2. **Positive Notum Liver X Adipose Pathways Enrichment File**
   - Notum正相关的脂肪组织基因
   - 用于通路富集分析：识别Notum上调的通路

3. **Negative Notum Liver X Adipose Pathways Enrichment File**
   - Notum负相关的脂肪组织基因
   - 用于通路富集分析：识别Notum下调的通路

4. **Absolute Notum Liver X Adipose Pathways Enrichment File**
   - Notum相关性最强的脂肪组织基因（不考虑方向）
   - 用于通路富集分析：识别Notum整体影响的通路

### 下一步分析建议

#### 1. 通路富集分析
使用生成的基因列表进行富集分析：
- **在线工具**：
  - DAVID (david.ncifcrf.gov)
  - Enrichr (enrichr.org)
  - Metascape (metascape.org)
- **R包**：
  - clusterProfiler
  - enrichR
  - ReactomePA

#### 2. 组织特异性验证
- 使用BioGPS检查候选基因的组织表达模式
- 确认基因在源组织高表达
- 网址：biogps.org

#### 3. 文献验证
- 在PubMed搜索候选基因的功能研究
- 验证其已知的内分泌功能
- 寻找潜在的机制研究

#### 4. 实验验证（如果可能）
- 体外实验：用重组蛋白处理脂肪细胞，检测靶基因表达
- 体内实验：构建基因敲除/过表达小鼠模型
- ELISA：测量血清中蛋白质水平

### 生物学意义

本分析流程能够：
1. **系统性识别**内分泌因子候选者
2. **量化评估**基因间的相关性强度
3. **预测通路**受特定分泌蛋白调控的生物学过程
4. **指导实验**为后续实验验证提供方向

### 方法学优势

1. **无偏好性**：不依赖先验知识，能发现新的内分泌因子
2. **统计稳健**：使用bicor相关性，对异常值不敏感
3. **可扩展性**：可应用于任何组织对
4. **整合性**：整合多个数据源（表达、蛋白注释等）

### 局限性

1. **相关性≠因果性**：需要实验验证
2. **数据依赖**：结果质量取决于输入数据质量
3. **组织特异性**：需要人工验证组织表达模式
4. **功能假设**：假设分泌蛋白能够到达目标组织并发挥作用

## 扩展应用

### 如何分析其他组织回路？

只需要更换输入的表达矩阵：

```R
# 例如：分析肠道到肌肉的回路
Intestine <- read.delim('Intestine.txt', check.names=F)
Muscle <- read.delim('Muscle.txt', check.names=F)

# 计算相关性
int.mus = bicorAndPvalue(Intestine, Muscle, use='pairwise.complete.obs')

# 后续步骤完全相同...
```

### 如何分析其他候选基因？

在步骤5.2中更改target_gene变量：

```R
# 例如：分析Fgf21（成纤维细胞生长因子21）
target_gene = 'Fgf21'
Fgf21 = filter(bicor.data, Gene_symbol_1 == target_gene) %>%
  select(-Gene_symbol_1) %>%
  rename(Gene_symbol = Gene_symbol_2)
# 然后继续后续分析...
```

### 批量分析多个基因

```R
# 分析Ssec排名前10的所有分泌蛋白
top_genes = head(Ssec$Gene_symbol, 10)

for(gene in top_genes) {
  print(paste("正在分析", gene, "..."))
  
  # 提取该基因的相关性
  gene_data = filter(bicor.data, Gene_symbol_1 == gene) %>%
    select(-Gene_symbol_1) %>%
    rename(Gene_symbol = Gene_symbol_2)
  
  # 生成通路富集文件
  upreg = gene_data %>% arrange(desc(bicor)) %>% head(500)
  write.table(upreg, 
              file = paste0("Positive_", gene, "_pathways.txt"),
              col.names = F, sep = '\t', quote = F)
}
```